## Import modules

In [1]:
import cobra
# from numba import jit
from tqdm import tqdm
import numpy as np
import math

## Model

In [20]:
model = cobra.io.read_sbml_model('Models/iAF1260.xml')
model

Name,iAF1260
Memory address,0x0109e4c860
Number of metabolites,1668
Number of reactions,2382
Objective expression,-1.0*BIOMASS_Ec_iAF1260_core_59p81M_reverse_3925e + 1.0*BIOMASS_Ec_iAF1260_core_59p81M
Compartments,"cytosol, periplasm, extracellular space"


## Elimination list and model config

In [21]:
elilist = model.exchanges
model.solver = 'glpk'

## Core model optimization

In [22]:
solWT = model.optimize()
solWT

,fluxes,reduced_costs
12DGR120tipp,0.000000,0.000000
12DGR140tipp,0.000000,-0.010031
12DGR141tipp,0.000000,0.000000
12DGR160tipp,0.000000,0.000000
12DGR161tipp,0.000000,0.000000
...,...,...
ZN2abcpp,0.000000,-0.008025
ZN2t3pp,0.000000,-0.002006
ZN2tpp,0.002327,0.000000
ZNabcpp,0.000000,-0.008025


## Objective value and fluxes

In [23]:
grWT = solWT.objective_value
J = solWT.fluxes

## Flux operations to obtain Jnz

In [24]:
Jnz_before_filtering = np.flatnonzero(J)
eliIdx = [model.reactions.index(reaction_id) for reaction_id in elilist]
Jnz = np.setdiff1d(Jnz_before_filtering,eliIdx)
Jnz

array([  68,   69,   70,   71,   72,   73,   74,   77,   78,   79,   87,
         88,   89,   90,   91,   92,   93,   96,   97,   98,   99,  100,
        101,  102,  103,  104,  105,  108,  109,  124,  152,  163,  164,
        165,  166,  167,  186,  190,  191,  192,  197,  198,  201,  202,
        206,  211,  222,  223,  224,  225,  245,  246,  249,  251,  252,
        253,  254,  255,  260,  263,  265,  304,  311,  312,  317,  335,
        336,  341,  348,  355,  356,  357,  358,  363,  374,  375,  376,
        381,  391,  394,  401,  414,  424,  425,  426,  446,  447,  450,
        451,  454,  455,  466,  481,  488,  492,  493,  506,  508,  516,
        523,  537,  541,  542,  548,  549,  553,  566,  572,  573,  574,
        582,  583,  584,  585,  587,  589,  590,  592,  593,  594,  596,
        597,  606,  607,  620,  621,  624,  625,  635,  643,  644,  648,
        649,  651,  653,  655,  657,  659,  661,  667,  669,  671,  694,
       1004, 1005, 1042, 1047, 1058, 1059, 1074, 10

## Single lethal reactions

In [25]:
Jsl_idx = []
for delIdx_i in Jnz:
    with model:
        model.reactions[delIdx_i].knock_out()
        solKO_i = model.slim_optimize()
        if solKO_i < 0.01 * grWT or math.isnan(solKO_i) == True:
            Jsl_idx.append(int(delIdx_i))
Jsl_idx = np.array(Jsl_idx)
Jsl_idx

array([  68,   70,   72,   74,   77,   78,   79,   87,   89,   90,   91,
         93,   96,   97,   98,   99,  101,  102,  103,  105,  108,  109,
        124,  152,  163,  164,  165,  167,  186,  190,  191,  192,  201,
        202,  222,  223,  224,  225,  245,  246,  249,  251,  252,  253,
        254,  260,  263,  265,  304,  311,  312,  317,  335,  336,  341,
        355,  356,  357,  363,  375,  381,  391,  394,  414,  424,  425,
        426,  446,  447,  454,  455,  481,  488,  492,  493,  506,  508,
        541,  542,  548,  549,  553,  566,  572,  573,  582,  583,  584,
        585,  587,  590,  592,  593,  594,  596,  597,  606,  620,  621,
        624,  625,  635,  643,  644,  648, 1004, 1047, 1107, 1133, 1137,
       1142, 1143, 1148, 1161, 1200, 1206, 1212, 1238, 1250, 1251, 1253,
       1254, 1298, 1314, 1328, 1329, 1358, 1361, 1382, 1383, 1390, 1395,
       1402, 1403, 1404, 1405, 1417, 1426, 1427, 1428, 1429, 1434, 1449,
       1450, 1451, 1452, 1455, 1456, 1457, 1458, 14

## Eliminating single lethal reactions to continue further

In [26]:
Jnz_copy = np.setdiff1d(Jnz,Jsl_idx)
Jnz_copy

array([  69,   71,   73,   88,   92,  100,  104,  166,  197,  198,  206,
        211,  255,  348,  358,  374,  376,  401,  450,  451,  466,  516,
        523,  537,  574,  589,  607,  649,  651,  653,  655,  657,  659,
        661,  667,  669,  671,  694, 1005, 1042, 1058, 1059, 1074, 1075,
       1103, 1113, 1129, 1162, 1164, 1196, 1197, 1198, 1211, 1232, 1234,
       1243, 1247, 1275, 1278, 1280, 1281, 1300, 1343, 1344, 1416, 1435,
       1448, 1460, 1474, 1578, 1621, 1648, 1681, 1699, 1700, 1712, 1730,
       1732, 1736, 1827, 1828, 1848, 1886, 1889, 1890, 1906, 1917, 1918,
       1919, 1920, 1922, 1959, 2019, 2025, 2026, 2032, 2066, 2068, 2078,
       2086, 2105, 2106, 2129, 2131, 2133, 2134, 2137, 2138, 2191, 2193,
       2202, 2244, 2245, 2258, 2260, 2268, 2271, 2345, 2352, 2379])

## Double and Triple lethal reactions

### First part

In [27]:
Jdl_idx = []
Jtl_idx = []

for delIdx_i in tqdm(Jnz_copy,desc='Part 1 of 2'):
    with model:
        model.reactions[delIdx_i].knock_out()
        solKO_i = model.optimize()
        newnnz = np.flatnonzero(solKO_i.fluxes)
        Jnz_i_before_filtering = np.setdiff1d(newnnz,Jnz)
        Jnz_i = np.setdiff1d(Jnz_i_before_filtering,eliIdx)

        for delIdx_j in Jnz_i:
            with model:
                model.reactions[delIdx_j].knock_out()
                solKO_ij = model.optimize()
                if solKO_ij.objective_value < 0.01 * grWT and solKO_ij.status != 'infeasible':
                    Jdl_idx.append([int(delIdx_i),int(delIdx_j)])
                    
                elif solKO_ij.status == 'infeasible':
                    solKO_ij_infeasibility = model.optimize()
                    
                    if solKO_ij_infeasibility.objective_value < 0.01 * grWT or math.isnan(solKO_ij_infeasibility.objective_value) == True:
                        Jdl_idx.append([int(delIdx_i),int(delIdx_j)])

                    Jnz_ij_before_filtering = np.flatnonzero(solKO_ij_infeasibility.fluxes)
                    Jnz_ij_after_filtering = np.setdiff1d(Jnz_ij_before_filtering,Jnz_before_filtering)
                    Jnz_ij = np.setdiff1d(Jnz_ij_after_filtering,eliIdx)
                    
                    for delIdx_k in Jnz_ij:
                        with model:
                            model.reactions[delIdx_k].knock_out()
                            solKO_ijk = model.slim_optimize()
                            if solKO_ijk < 0.01 * grWT or math.isnan(solKO_ijk) == True:
                                Jtl_idx.append([int(delIdx_i),int(delIdx_j),int(delIdx_k)])

Part 1 of 2: 100%|██████████| 120/120 [02:40<00:00,  1.34s/it]


### Second part

In [28]:
for delIdx_i in tqdm(Jnz_copy,desc='Part 2 of 2'):
    for delIdx_j in Jnz_copy:
        if np.where(Jnz_copy==delIdx_j) < np.where(Jnz_copy==delIdx_i):
            with model:
                model.reactions[delIdx_i].knock_out()
                model.reactions[delIdx_j].knock_out()
                solKO_ij = model.slim_optimize()
                if solKO_ij < 0.01 * grWT:
                    Jdl_idx.append([int(delIdx_i),int(delIdx_j)])
                
                elif math.isnan(solKO_ij) == True:
                    solKO_ij = model.optimize()
                    if solKO_ij.objective_value < 0.01 * grWT or math.isnan(solKO_ij.objective_value) == True:
                        Jdl_idx.append([int(delIdx_i),int(delIdx_j)])

                    Jnz_ij_before_filtering = np.flatnonzero(solKO_ij.fluxes)
                    Jnz_ij_after_filtering = np.setdiff1d(Jnz_ij_before_filtering,Jnz_before_filtering)
                    Jnz_ij = np.setdiff1d(Jnz_ij_after_filtering,eliIdx)

                    for delIdx_k in Jnz_ij:
                        with model:
                            solKO_ijk = model.slim_optimize()
                            if solKO_ijk < 0.01 * grWT or math.isnan(solKO_ijk) == True:
                                Jtl_idx.append([int(delIdx_i),int(delIdx_j),int(delIdx_k)])

                    for delIdx_k in Jnz_copy:
                        with model:
                            if np.where(Jnz_copy==delIdx_k) < np.where(Jnz_copy==delIdx_j):
                                solKO_ijk = model.slim_optimize()
                                if solKO_ijk < 0.01 * grWT or math.isnan(solKO_ijk) == True:
                                    Jtl_idx.append([int(delIdx_i),int(delIdx_j),int(delIdx_k)])

# Eliminate double lethal reaction deletions in triple lethal reacutions
Jdl_idx = np.array(Jdl_idx)
Jtl_idx = np.array(Jtl_idx)

Part 2 of 2: 100%|██████████| 120/120 [03:28<00:00,  1.74s/it]


In [17]:
Jdl_idx

array([[ 166,  199],
       [ 348,  347],
       [ 466,  123],
       [ 466,  467],
       [ 537,  538],
       [ 588,  589],
       [ 607, 1447],
       [ 649,  650],
       [ 651,  652],
       [ 653,  654],
       [ 655,  656],
       [ 657,  658],
       [ 659,  660],
       [ 661,  662],
       [ 667,  668],
       [ 669,  670],
       [ 671,  672],
       [ 694, 2036],
       [1074,  578],
       [1074,  579],
       [1074,  695],
       [1074,  697],
       [1074, 1087],
       [1074, 1088],
       [1074, 1089],
       [1074, 1090],
       [1074, 1091],
       [1074, 2151],
       [1129, 2189],
       [1162, 1707],
       [1196, 2036],
       [1198, 1102],
       [1198, 1348],
       [1198, 1349],
       [1243, 1707],
       [1247, 1252],
       [1259, 1278],
       [1259, 1280],
       [1435, 1406],
       [1435, 2364],
       [1448, 1447],
       [1460, 1459],
       [1474, 1473],
       [1681, 1686],
       [1730,  207],
       [1848, 1849],
       [1886, 1315],
       [1886,

In [19]:
Jtl_idx

(0,)

In [ ]:
temporary = []
g = np.zeros(Jdl_idx.shape[0])
for delIdx_i in Jtl_idx:
    for delIdx_j in Jdl_idx:
        g[np.where(Jdl_idx==delIdx_j)[0][0]] = np.sum(np.in1d(delIdx_i,delIdx_j))
#         if g[np.where(Jdl_idx==delIdx_j)[0][0]] >= 2:
#             break
#             print(g[np.where(Jdl_idx==delIdx_j)[0][0]])
    if np.max(g) < 2:
#         print(g[np.where(Jdl_idx==delIdx_j)[0][0]])
        temporary.append(delIdx_i)

# Jtl_idx_copy = np.array(temporary)
# Jtl_idx_copy = np.unique(np.sort(Jtl_idx_copy), axis=0)

#     Jdl = [model.reactions.get_by_any(rxn_pair_idx) for rxn_pair_idx in Jdl_idx]
#     Jtl = [model.reactions.get_by_any(rxn_triplet_idx) for rxn_triplet_idx in Jtl_idx]

In [ ]:
Jdl_idx.shape

In [ ]:
np.unique(np.sort(np.array(temporary), axis=1), axis=0).shape

In [ ]:
np.savetxt('Jtl.csv',Jtl_idx,delimiter=',')

In [ ]:
Jsl_genes = []
for reaction in Jsl_idx:
    print(model.genes.index(model.reactions[reaction].genes))